In [1]:
import pandas as pd  
import numpy as np
import tensorflow as tf  
from tensorflow import keras  
from tensorflow.keras.layers import Conv2D, BatchNormalization  
from tensorflow.keras.layers import Activation, MaxPooling2D  
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import UpSampling2D  
from tensorflow.keras.layers import Input, Flatten, Dense  
from tensorflow.keras.models import Model 
import numpy as npdata
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from math import ceil

In [2]:
DATASET_B_PATH_WIN = 'F:\大学\第40期PRP\特征提取\\1_feature_analysis\Intergated-DATASET-D.csv'
PREPROCESS_PATH_WIN = 'F:/大学/第40期PRP/PRP 测试代码/CNN/congestion.csv'
data_in = pd.read_csv(DATASET_B_PATH_WIN).drop(['Unnamed: 0'], axis=1)

In [3]:
data = data_in.loc[data_in.date <= 20161105]

In [4]:
ROW_LIM_DOWN = int(data.row_id.min())
ROW_LIM_UP = int(data.row_id.max() - 109)
COL_LIM_DOWN = int(data.col_id.min())
COL_LIM_UP = int(data.col_id.max() - 109)
print((ROW_LIM_DOWN, ROW_LIM_UP, COL_LIM_DOWN, COL_LIM_UP))

(-139, -110, 0, 29)


In [5]:
# 选定待处理的空间网格区域
data = data.loc[(data.row_id >= ROW_LIM_DOWN)&(data.row_id <= ROW_LIM_UP) &(data.col_id >= COL_LIM_DOWN) & (data.col_id <= COL_LIM_UP)]
# 将所有网格id转换成int型
for c in ['row_id', 'col_id', 'time_id']:  
    data[c] = data[c].astype(int)
# 将date和id处理成正整数，然后排序
data['date'] -= data['date'].min()
data['row_id'] -= data['row_id'].min()
data['col_id'] -= data['col_id'].min()
data['time_id'] -= data['time_id'].min()
data = data.sort_values(['date', 'row_id', 'col_id', 'time_id']).reset_index(drop=True)

In [6]:
data

,row_id,col_id,time_id,aveSpeed,gridAcc,volume,speedStd,stopNum,date
0,0,5,9,15.571961,-2.323410,1,0.000000,0.000000,0
1,0,5,12,10.679653,-0.098228,1,0.000000,0.000000,0
2,0,5,15,12.348424,0.018499,2,1.345171,0.000000,0
3,0,5,18,9.483641,0.109564,1,0.000000,0.000000,0
4,0,5,24,12.754410,-0.000014,1,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...
318101,29,29,265,9.276823,-0.714574,8,5.971452,1.125000,4
318102,29,29,266,8.506305,-0.687411,14,5.056376,0.928571,4
318103,29,29,267,7.579736,-0.116602,11,5.190623,2.363636,4
318104,29,29,268,10.921124,-0.013864,15,2.878470,0.000000,4


In [7]:
def grid_recovery(df_, cols=[], lens=[]):
    df = df_.copy()
    lcols = len(cols)
    llens = len(lens)
    
    if lcols != llens: # 确保输入的网格名称和网格长度信息的长度一致
        raise ValueError(f'Lengths of cols ({lcols}) and lens ({llens}) mismatch.')
    
    recovery_df = None
    for c, l in zip(cols, lens):
        tmp_df = pd.DataFrame({c:range(l)})
        tmp_df['flag'] = True
        if recovery_df is None:
            recovery_df = tmp_df.copy()
        else:
            recovery_df = recovery_df.merge(tmp_df, 'left', 'flag')
    
    del recovery_df['flag']
    
    df = pd.merge(recovery_df, df, on=['date','row_id', 'col_id', 'time_id'], how='outer')
    df = df.fillna(0)
    return df

In [8]:
NROWS = int(data.row_id.max() - data.row_id.min() + 1) # 空间网格行数  
NCOLS = int(data.col_id.max() - data.col_id.min() + 1) # 空间网格列数  
NTIME = int(data.time_id.max() - data.time_id.min() + 1) # 时间网格数  
NDATE = 5 # 日期网格数
time_density = 6
space_density = 2
win_s = 10

In [9]:
data = grid_recovery(data, ['date', 'row_id', 'col_id', 'time_id'], [NDATE, NROWS, NCOLS, NTIME])  
for c in ['volume', 'stopNum']:  
    data[c] = data[c].astype(int) # 调整数据类型 

In [10]:
data

,date,row_id,col_id,time_id,aveSpeed,gridAcc,volume,speedStd,stopNum
0,0,0,0,0,0.000000,0.000000,0,0.000000,0
1,0,0,0,1,0.000000,0.000000,0,0.000000,0
2,0,0,0,2,0.000000,0.000000,0,0.000000,0
3,0,0,0,3,0.000000,0.000000,0,0.000000,0
4,0,0,0,4,0.000000,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...,...,...
1214995,4,29,29,265,9.276823,-0.714574,8,5.971452,1
1214996,4,29,29,266,8.506305,-0.687411,14,5.056376,0
1214997,4,29,29,267,7.579736,-0.116602,11,5.190623,2
1214998,4,29,29,268,10.921124,-0.013864,15,2.878470,0


In [11]:
[NDATE, NROWS, NCOLS, NTIME]

[5, 30, 30, 270]

In [12]:
# 当前网格数太多了，可以进一步合并
data['hourid'] = data['time_id'] // time_density # 合并时间网格  
data['new_rowid'] = data.row_id // space_density# 合并空间网格  
data['new_colid'] = data.col_id //space_density
congest = data.groupby( # 计算合并网格后各网格的流量  
    ['date', 'new_rowid', 'new_colid', 'hourid']).aveSpeed.mean().reset_index()  
congest.columns = ['date', 'row_id', 'col_id', 'hourid', 'aveSpeed'] 

TIME_UNIT = data.groupby('hourid').size().count() # 每个date里所包含的hourid的数量
from math import ceil

congest_pivot = congest.pivot_table(  
    index=['date', 'hourid', 'row_id'],  
    columns='col_id',  
    values='aveSpeed').fillna(0).reset_index() # 网格转换  
congest_pivot['timeseq'] = congest_pivot['date'] * TIME_UNIT + congest_pivot['hourid'] # 时间序号  
congest_pivot_np = congest_pivot[[c for c in range(ceil(NCOLS/space_density))]].values # 提取速度数值

def gen_movie(df, win_t, win_T, nrows=ceil(NROWS/space_density), ncols=ceil(NCOLS/space_density), win_s=win_s, ntime=NDATE*TIME_UNIT):  #注意：win_s必须得小于nrows,ncols!!!
    n_i = nrows - win_s + 1   
    n_j = ncols - win_s + 1   
    piece = []  
    for t in range(7*TIME_UNIT - 1, ntime):   #第一天末尾的hourid索引实际上是TIME_UNIT - 1，而不是TIME_UNIT
        for i in range(n_i):  
            for j in range(n_j):  
                # 周期特征  
                prd_piece = df[t-(win_T*TIME_UNIT - 1) : t - (TIME_UNIT - 1) + 1: TIME_UNIT, i:i+win_s, j:j+win_s]  
                # 邻近特征  
                nbr_piece = df[t-win_t:t+1, i:i+win_s, j:j+win_s]  
                piece.append(np.vstack([prd_piece, nbr_piece]))  
    return np.stack(piece)  

    


In [13]:
# 卷积层
def conv_layer(inputs,  
             num_filters=16,  
             kernel_size=3,  
             strides=1,  
             data_format='channels_first',  
             activation='relu',  
             batch_normalization=True,  
             maxpooling=True,  
             pool_size=2,  
             pool_strides=2):  
    conv = Conv2D(num_filters,  
                 kernel_size=kernel_size,  
                 strides=strides,  
                 padding='same',  
                 data_format=data_format,  
                 kernel_regularizer=l2(1e-4))  
    x = conv(inputs)  # 卷积
    if batch_normalization:
        x = BatchNormalization()(x) # 批归一化
    if activation is not None:
        x = Activation(activation)(x) # 激活函数
    if maxpooling:
        x = MaxPooling2D(pool_size=pool_size,  
                        strides=pool_strides,  
                        data_format=data_format,  
                        padding='same')(x) # 池化
    return x

def upsample_layer(inputs,  
                up_size=2,  
                interpolation='nearest',  
                data_format='channels_first'):
    upsample = UpSampling2D(size=up_size,
                    data_format=data_format,
                    interpolation=interpolation)  
    x = upsample(inputs) # 上采样
    return x

def cnn_model(input_shape):  
    inputs = Input(shape=input_shape)  
    x = conv_layer(inputs, 16, pool_strides=1)  
    x = conv_layer(x, 32, pool_strides=1)  
    x = conv_layer(x, 32, pool_strides=1)  
    x = conv_layer(x, 16)  
    x = upsample_layer(x, 2)  
    x = conv_layer(x, 1, maxpooling=False)  
    y = Flatten(data_format='channels_first')(x)  
    y = Dense(128, activation='relu')(y)  
    y = Dense(128, activation='relu')(y)  
    outputs = Dense(100, activation='relu')(y)  
    # 建立模型  
    model = Model(inputs=inputs, outputs=outputs)  
    return model

In [14]:
N_SAMPLE_PER_HOUR = (ceil(NROWS/space_density) - win_s + 1) * (ceil(NCOLS/space_density) - win_s + 1) # 计算数据增强（裁剪）之后，一个hourid内的数据数量
N_SAMPLES_LAST_DAY = TIME_UNIT * N_SAMPLE_PER_HOUR #计算最后一天的样本（数据）量

In [15]:

Win_t = [1,2,3]
Win_T = [1,2,3]

In [16]:
ex_records = []
ex_data = []

In [17]:
for win_t in Win_t:
    for win_T in Win_T:

        movie = gen_movie(np.asarray(congest_pivot_np.reshape(NDATE*TIME_UNIT, ceil(NROWS/space_density), ceil(NCOLS/space_density)), order='C'), win_t=win_t, win_T=win_T)  

        data_x = movie[:, :win_t+win_T].astype('float32')
        data_y = movie[:, win_t+win_T].astype('float32')
        Max = max(data_x.max(), data_y.max())
        data_x /= Max # 归一化  
        data_y /= Max # 归一化 


        train_x, train_y = data_x[:-N_SAMPLES_LAST_DAY], data_y[:-N_SAMPLES_LAST_DAY] # 训练集     
        test_x, test_y = data_x[-N_SAMPLES_LAST_DAY:], data_y[-N_SAMPLES_LAST_DAY:] # 测试集
        model = cnn_model(train_x.shape[1:])      
        opt = keras.optimizers.Adam(learning_rate=2e-4)      
        model.compile(loss='mse',      
                optimizer=opt,      
                metrics=['mae', 'mse'])      
        batch_size = 64 # 训练批次大小    
        epochs = 50  # 训练轮数    
        earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) # 早停策略    
        model.fit(train_x, train_y.reshape(-1, 100),      
                batch_size=batch_size,      
                epochs=epochs,       
                validation_split=0.2,      
                callbacks=[earlystop],      
                shuffle=True) # 模型训练

        predictions_test = model.predict(test_x, batch_size=512)    
        mae_test = mean_absolute_error(predictions_test * Max, test_y.reshape(-1, 100) * Max)    
        mse_test = mean_squared_error(predictions_test * Max, test_y.reshape(-1, 100) * Max)  
        ex_records.append(f'win_t: {win_t}, win_T: {win_T}, MAE: {mae_test}, MSE: {mse_test}, RMSE: {np.sqrt(mse_test)}')
        ex_data.append((mae_test, mse_test, np.sqrt(mse_test)))

            
            

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1324 samples, validate on 332 samples
Epoch 1/50
1324/1324 [==============================] - 5s 3ms/sample - loss: 0.0605 - mean_absolute_error: 0.1576 - mean_squared_error: 0.0525 - val_loss: 0.0608 - val_mean_absolute_error: 0.1623 - val_mean_squared_error: 0.0528
Epoch 2/50
1324/1324 [==============================] - 0s 222us/sample - loss: 0.0471 - mean_absolute_error: 0.1325 - mean_squared_error: 0.0391 - val_loss: 0.0585 - val_mean_absolute_error: 0.1592 - val_mean_squared_error: 0.0506
Epoch 3/50
1324/1324 [==============================] - 0s 230us/sample - loss: 0.0433 - mean_absolute_error: 0.1292 - mean_squared_error: 0.0354 - val_loss: 0.0555 - val_mean_absolute_error: 0.1550 - val_mean_squared_error: 0.0477
Epoch 4/50
1324/1324 [==============================] - 0s 206us/sample - loss: 0.0406 - mean_absolute_error: 0.1273 - mean_squared_error: 0.0328 - val_loss: 0.0528 - val_mean_ab

In [18]:
for record in ex_records:
    print(record)

win_t: 1, win_T: 1, MAE: 0.9605603814125061, MSE: 2.6393401622772217, RMSE: 1.62460458278656
win_t: 1, win_T: 2, MAE: 0.9284742474555969, MSE: 2.6267669200897217, RMSE: 1.6207304000854492
win_t: 1, win_T: 3, MAE: 0.9544178247451782, MSE: 2.6659839153289795, RMSE: 1.632784128189087
win_t: 2, win_T: 1, MAE: 1.0005710124969482, MSE: 3.2193222045898438, RMSE: 1.794247031211853
win_t: 2, win_T: 2, MAE: 0.9457591772079468, MSE: 2.567530870437622, RMSE: 1.6023516654968262
win_t: 2, win_T: 3, MAE: 0.8961565494537354, MSE: 2.327275037765503, RMSE: 1.5255409479141235
win_t: 3, win_T: 1, MAE: 0.9319643378257751, MSE: 2.6751861572265625, RMSE: 1.6355996131896973
win_t: 3, win_T: 2, MAE: 0.9244349002838135, MSE: 2.5859506130218506, RMSE: 1.6080890893936157
win_t: 3, win_T: 3, MAE: 0.8382073044776917, MSE: 1.8866239786148071, RMSE: 1.3735443353652954


In [19]:
ex_data

[(0.9605604, 2.6393402, 1.6246046),
 (0.92847425, 2.626767, 1.6207304),
 (0.9544178, 2.665984, 1.6327841),
 (1.000571, 3.2193222, 1.794247),
 (0.9457592, 2.5675309, 1.6023517),
 (0.89615655, 2.327275, 1.525541),
 (0.93196434, 2.6751862, 1.6355996),
 (0.9244349, 2.5859506, 1.6080891),
 (0.8382073, 1.886624, 1.3735443)]